---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
homes = pd.read_csv('City_Zhvi_AllHomes.csv')
homes.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [19]:
gdp_xl = pd.ExcelFile("gdplev.xls")
gdp = gdp_xl.parse(gdp_xl.sheet_names[0], skiprows=213, header=6)
gdp.dropna(axis=1,inplace=True)
gdp.columns=['Quarter','Current Dollar','2009 Dollar']
gdp.head()

,Quarter,Current Dollar,2009 Dollar
0,2000q1,10031.0,12359.1
1,2000q2,10278.3,12592.5
2,2000q3,10357.4,12607.7
3,2000q4,10472.3,12679.3
4,2001q1,10508.1,12643.3


In [3]:
def get_state_region(indices,df):
    filtered_df = pd.DataFrame(columns=['State','RegionName'])
    for index in indices:
        state = df.iloc[index].values[0][0:df.iloc[index].values[0].index(df.iloc[index].values[0][df.iloc[index].values[0].find("[")])]
        for index, row in (df.iloc[(index+1):]).iterrows():
            if (~("[edit]" in row['data']) ) == -1:
                    if(("(" in row['data']) == True):
                        paren_index = row['data'].index(row['data'][row['data'].find("(")])
                        filtered_df.loc[len(filtered_df)] = state, row['data'][0:paren_index-1]
                    else:
                        filtered_df.loc[len(filtered_df)] = state, row['data']                  
            else:
                break
    #print(len(filtered_df))       
    return filtered_df

In [170]:
university_towns_df = pd.read_csv('university_towns.txt', sep="\n", header=None)
university_towns_df.columns = ['data']

In [5]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
homes['StateFull'] = homes.State.map(states)
homes.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,StateFull
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,576200,578400,582200,588000,592200,592500,590200,588000,586400,New York
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,560800,562800,565600,569700,574000,577800,580600,583000,585100,California
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,206900,206200,205800,206200,207300,208200,209100,211000,213000,Illinois
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,121600,121800,123300,125200,126400,127000,127400,128300,129100,Pennsylvania
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,185300,186600,188000,189100,190200,191300,192800,194500,195900,Arizona


In [7]:
def get_list_of_university_towns():
    state_indices = university_towns_df[university_towns_df['data'].str.contains('\[edit\]')].index
    university_towns_final_df = pd.DataFrame(columns=['State','RegionName'])
    #for state_index in state_indices:
    university_towns_final_df = get_state_region(state_indices, university_towns_df)
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return university_towns_final_df

In [94]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    i = 0

    for index, row in (gdp).iterrows():
        if(i> 0):
            if(((gdp.iloc[i]['Current Dollar'] < gdp.iloc[i-1]['Current Dollar']) and (gdp.iloc[i+1]['Current Dollar'] < gdp.iloc[i]['Current Dollar'])) == True):
                break
        i = i + 1
    return str(gdp.iloc[i-1:i].Quarter.values[0])
get_recession_start()

'2008q3'

In [95]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    i = 0

    for index, row in (gdp).iterrows():
        if(i> 0):
            if(((gdp.iloc[i]['Current Dollar'] < gdp.iloc[i-1]['Current Dollar']) and (gdp.iloc[i+1]['Current Dollar'] < gdp.iloc[i]['Current Dollar'])) == True):
                break
        i = i + 1
    j = i   
    for index, row in (gdp[i:]).iterrows():
        if(j> 0):
            if(((gdp.iloc[j]['Current Dollar'] > gdp.iloc[j-1]['Current Dollar']) and (gdp.iloc[j+1]['Current Dollar'] > gdp.iloc[j]['Current Dollar'])) == True):
                break
        j = j + 1
    return str(gdp.iloc[j+1:j+2].Quarter.values[0])
get_recession_end()

'2009q4'

In [96]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    i = 0

    for index, row in (gdp).iterrows():
        if(i> 0):
            if(((gdp.iloc[i]['Current Dollar'] < gdp.iloc[i-1]['Current Dollar']) and (gdp.iloc[i+1]['Current Dollar'] < gdp.iloc[i]['Current Dollar'])) == True):
                break
        i = i + 1
    j = i   
    for index, row in (gdp[i:]).iterrows():
        if(j> 0):
            if(((gdp.iloc[j]['Current Dollar'] > gdp.iloc[j-1]['Current Dollar']) and (gdp.iloc[j+1]['Current Dollar'] > gdp.iloc[j]['Current Dollar'])) == True):
                break
        j = j + 1   
    return gdp.loc[gdp.iloc[i:j+2]['Current Dollar'].idxmin()].Quarter
get_recession_bottom()

'2009q2'

In [169]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    starting = 51
    qtrno = 1
    qtr_df = pd.DataFrame(columns=['State','RegionName'])
    qtr_df[['State','RegionName']] = homes[['StateFull','RegionName']]
    i = 0
    for qtr in range(0,int(len(homes.columns[starting:])/3)):
        qtrname = "{}q{}".format(int(homes.columns[starting].split('-')[0]),qtrno)
        #print(i)
        if(qtr == 66):
            qtr_df[qtrname] = pd.Series (row.mean() for index, row in (homes[homes.columns[starting:starting+2]]).iterrows())   
        else:
            qtr_df[qtrname] = pd.Series (row.mean() for index, row in (homes[homes.columns[starting:starting+3]]).iterrows())
        if(qtrno >= 4):
            qtrno = 1
        else:
            qtrno = qtrno + 1
        starting = starting + 3
        i = i + 1
    qtr_df.set_index(['State','RegionName'], inplace=True)

    
    return qtr_df
convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]  

202266.66666666666

In [267]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    hdf = convert_housing_data_to_quarters()
    rec_start=get_recession_start()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()
    qtr_bfr_rec_start = hdf.columns[(list(hdf.columns).index(rec_start)-1)]
    hdf['PriceRatio'] = hdf[qtr_bfr_rec_start].div(hdf[rec_bottom])

    df = pd.merge(hdf.reset_index(), ul, on=ul.columns.tolist(),
                 indicator='_flag', how='outer')
    uni_df = df[df['_flag'] == 'both']
    non_uni_df = df[df['_flag'] != 'both']
    #len(non_uni_df)
    #non_uni_df = non_uni_df.iloc[]
    non_uni_df = non_uni_df[~pd.isnull(non_uni_df['PriceRatio'])]
    uni_df = uni_df[~pd.isnull(uni_df['PriceRatio'])]
    ttest = ttest_ind(uni_df['PriceRatio'],non_uni_df['PriceRatio'])
    different = False
    p = ttest[1]
    if(p < 0.01):
        different = True
    better = "non-university town"    
    if(uni_df['PriceRatio'].mean() < non_uni_df['PriceRatio'].mean()): 
        better = "university town"
        
    return (different,p,better)

run_ttest()

(True, 0.0027240637047614607, 'university town')